
 Use Jump CP metadata file as input 

    - Add Smiles to the metadata file, remove all rows with invalid InChI or SMILES fingerprint
    - Compute Pharmacophores for each compound
    - Save pharmacophore vectors as sparse file
    

# Initialization

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import sys
import argparse
import csv
import json
import time
import types, copy, pprint
from datetime import datetime
for p in ['./src','../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
print(sys.path)
from operator import  mod
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
# from utils_chem import getTPSAFromRDKit, getSmilesFromRDKit
from KevinsRoutines.utils.utils_chem  import getTPSAFromRDKit, getSmilesFromRDKit
# appendSmilesFromRDKit,
from KevinsRoutines.utils.utils_chem  import (getPharmacophoresFromRDKit,  appendMolFromRDKit, 
                                              my_enumerator, starmap_with_kwargs_async, fingerprint_to_sparse_matrix)
# from KevinsRoutines.utils.utils_general  import  list_namespace 

from itertools import repeat
import more_itertools
from multiprocessing import Pool, process
from os import getpid
from rdkit.Chem import PandasTools

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=10, infstr='inf', linewidth=150, nanstr='nan')
pd.options.display.width = 180

['../..', './src', '/home/kevin/WSL-shared/Cellpainting/cj-datasets', '/home/kevin/miniforge3/envs/cp311/lib/python311.zip', '/home/kevin/miniforge3/envs/cp311/lib/python3.11', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg']


In [3]:
time.strftime('%X %x %Z')
# datetime.now().strftime('%X.%f')

'18:07:15 05/07/24 CEST'

In [5]:
import jcp_datasets as ds

In [9]:
print(f" prefix                                                  : {ds.prefix} \n" )
print(f" cmpnd Metadata Input File                               : {ds.compoundMetadataInputFile} \n")
print()
print(f" compound Metadata : ")
print(f" ------------------- ")
print(f" Metadata Input File                         : {ds.compoundMetadataInputFile}")
print(f"       + Smiles                              : {ds.compoundMetadataSmilesFile}")
print(f"       + Smiles Cleaned                      : {ds.compoundMetadataSmilesCleanFile}\n")
print(f"       + TPSA                                : {ds.compoundMetadataTPSAFile}")
print(f"       + TPSA Cleaned (removed TPSA = 0)     : {ds.compoundMetadataTPSACleanFile}\n")
print(f"       + Base64 Pharmacophores               : {ds.compoundMetadataPharmacophoreFile}\n")

print(f" compound/TPSA File:")
print(f" -------------------")
print(f" compound TPSA File                          : {ds.compoundTPSAFile}")
print(f" compound TPSA Clean File                    : {ds.compoundTPSACleanFile}\n")

print(f" compound/Pharmacophore Files :")
print(f" ------------------------------")
print(f" compound Pharmacophore Sparse File          : {ds.compoundPharmacophoreFile}")
print(f" compound Pharmacophore Sparse Cleaned File  : {ds.compoundPharmacophoreCleanFile}\n")
print(f" compound Pharmacophore Dense File           : {ds.compoundPharmacophoreDenseFile}")
print(f" compound Pharmacophore Dense Zipped File    : {ds.compoundPharmacophoreDenseZipFile}")

 prefix                                                  :  

 cmpnd Metadata Input File                               : ./metadata/_compound_metadata.csv 


 compound Metadata : 
 ------------------- 
 Metadata Input File                         : ./metadata/_compound_metadata.csv
       + Smiles                              : ./output_11102023/compound_metadata_smiles.csv
       + Smiles Cleaned                      : ./output_11102023/compound_metadata_smiles_clean.csv

       + TPSA                                : ./output_11102023/compound_metadata_tpsa.csv
       + TPSA Cleaned (removed TPSA = 0)     : ./output_11102023/compound_metadata_tpsa_clean.csv

       + Base64 Pharmacophores               : ./output_11102023/_compound_metadata_pcfp.csv

 compound/TPSA File:
 -------------------
 compound TPSA File                          : ./output_11102023/compound_TPSA.csv
 compound TPSA Clean File                    : ./output_11102023/compound_TPSA_clean.csv

 compound/Pharmacophor

# Append pharmacophore fingerprints to dataframe 

- using multiparallelism and callign `getPharmacophoresFromRDKit`**

## Read Jump-CP compound metadata (+ smiles) file

In [5]:
# print(f" prefix                         : {prefix}" )
# print(f" compound Metadata Input File   : {compoundMetadataInputFile}")
# print(f" compound Metadata Output File 1: {compoundMetadataSmilesFile}")
 
print()
print(f" compound Metadata + Smiles & TPSA (clean)            : {compoundMetadataTPSACleanFile}")
print(f"                   + Base64 Pharmacophores            : {compoundMetadataPharmacophoreFile}\n")
print(f" compound Pharmacophore Sparse File                   : {compoundPharmacophoreFile}")
print(f" compound Pharmacophore Sparse Cleaned File           : {compoundPharmacophoreCleanFile}")
print(f" compound Dense File                                  : {compoundPharmacophoreDenseFile}")
print(f" compound Dense Zipped File                           : {compoundPharmacophoreDenseZipFile}")
# print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")


 compound Metadata + Smiles & TPSA (clean)            : ./output_09112023/cp_compound_metadata_tpsa_clean.csv
                   + Base64 Pharmacophores            : ./output_09112023/cp_compound_metadata_pcfp.csv

 compound Pharmacophore Sparse File                   : ./output_09112023/cp_compound_pcfp_sparse.pkl
 compound Pharmacophore Sparse Cleaned File           : ./output_09112023/cp_compound_pcfp_sparse_clean.pkl
 compound Dense File                                  : ./output_09112023/cp_compound_pcfp_dense.npy
 compound Dense Zipped File                           : ./output_09112023/cp_compound_pcfp_dense.npz


In [6]:
 
df = pd.read_csv(compoundMetadataTPSACleanFile)

In [7]:
df.info()
df.head(3)
df.tail(3)
df[df['smiles'].isna()]
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116691 entries, 0 to 116690
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   JCP2022    116691 non-null  object 
 1   InChIKey   116691 non-null  object 
 2   InChI      116691 non-null  object 
 3   smiles     116691 non-null  object 
 4   TPSA       116691 non-null  float64
 5   lnTPSA     116691 non-null  float64
 6   log10TPSA  116691 non-null  float64
dtypes: float64(3), object(4)
memory usage: 6.2+ MB


,JCP2022,InChIKey,InChI,smiles,TPSA,lnTPSA,log10TPSA
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...,63.49,4.150882,1.802705
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,OC1=NCCCN1Cc2ccc(Cl)cc2,35.83,3.578786,1.554247
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4,73.72,4.300274,1.867585


,JCP2022,InChIKey,InChI,smiles,TPSA,lnTPSA,log10TPSA
116688,JCP2022_116751,ZZZUOLMMTJKOGE-UHFFFAOYSA-N,InChI=1S/C19H17N5O2/c1-23-9-6-13-7-10-24(19(26...,CN4C=CC3C=CN(CCN=C(O)c2cnc1ccccc1n2)C(=O)C=34,85.30,4.446174,1.930949
116689,JCP2022_116752,ZZZZIBSVRUABIA-UHFFFAOYSA-N,InChI=1S/C19H13F6N3O2S/c1-11-26-6-5-17(27-11)1...,Cc1nccc(n1)c2cccc(c2)NS(=O)(=O)c3cc(cc(c3)C(F)...,71.95,4.275971,1.857031
116690,JCP2022_116753,ZZZZVNLQWWXZTQ-UHFFFAOYSA-N,InChI=1S/C14H20N2O3/c1-18-13-5-3-2-4-12(13)14(...,COc1ccccc1C(O)=NCCN2CCOCC2,54.29,3.994340,1.734720


,JCP2022,InChIKey,InChI,smiles,TPSA,lnTPSA,log10TPSA


116691

## Calculate pharmacophoes using multiprocessing 

In [11]:
nrows = 60000
chunksize = 10000
progress_int = 500
processes = 6
idx = my_enumerator()
fn = getPharmacophoresFromRDKit
kwargs = dict({'append_pharmacophore': True, 'progress_int':progress_int}) 

print(f"Read metadata file : {compoundMetadataTPSACleanFile}")

Read metadata file : ./output_09112023/cp_compound_metadata_tpsa_clean.csv


In [9]:
df_iterator  = pd.read_csv(compoundMetadataTPSACleanFile , nrows=nrows, header=0,  chunksize= chunksize)

In [10]:
# dd = next(df_iterator)
# type(dd)
# dd.info()
# dd.head(4)

In [ ]:

args = zip(idx, df_iterator)

In [ ]:
# result = pool.starmap_async(get_pharmacophores, enumerate(df_iterator)) 

results = starmap_with_kwargs_async( fn=fn, args=args, kwargs = kwargs, processes = processes)


In [14]:
results.ready()

False

### Wait for completion of all processes

In [15]:
#     results.wait()
sleep_interval = 60
while not results.ready():
    print(f" {datetime.now().strftime('%X.%f')} | still waiting - - sleep for {sleep_interval} seconds")
    time.sleep(sleep_interval)
else:
    results = results.get()
    finish_time = time.perf_counter()
    print(f" {datetime.now().strftime('%X.%f')} | starmap_with_kwargs_async() | Results ready. . .  ")    
    print(f" {datetime.now().strftime('%X.%f')} | All processes complete")
    print(f" {datetime.now().strftime('%X.%f')} | Program finished in {finish_time-start_time:5f} seconds")  
    
print(f" {datetime.now().strftime('%X.%f')} | Process Ended ")    

 14:11:40.922127 | still waiting - - sleep for 60 seconds
 14:12:40.981142 | still waiting - - sleep for 60 seconds
 14:13:41.041456 | still waiting - - sleep for 60 seconds
 14:14:41.101130 | still waiting - - sleep for 60 seconds
 14:15:41.161379 | still waiting - - sleep for 60 seconds
 14:16:41.221662 | still waiting - - sleep for 60 seconds
 14:17:41.281900 | still waiting - - sleep for 60 seconds
 14:18:41.342212 | still waiting - - sleep for 60 seconds
 14:19:41.401147 | still waiting - - sleep for 60 seconds
 14:20:41.461376 | still waiting - - sleep for 60 seconds
 14:21:41.521759 | starmap_with_kwargs_async() | Results ready. . .  
 14:21:41.521912 | All processes complete
 14:21:41.521923 | Program finished in 637.154420 seconds
 14:21:41.522062 | Process Ended 


### check results 

In [23]:
type(results), len(results), len(results[0])

(list, 6, 3)

In [25]:
print(results[1][0])

1


In [26]:
print(results[1][1])

              JCP2022                     InChIKey                                              InChI                                             smiles   TPSA    lnTPSA  \
10000  JCP2022_010005  CBFNFTQOUVNGIF-UHFFFAOYSA-N  InChI=1S/C18H18N6O/c1-13-20-21-22-24(13)17(14-...         CC1=NN=NN1C(C(=O)N2CC(C2)c3ccccn3)c4ccccc4  76.80  4.341205   
10001  JCP2022_010006  CBFNQLBICBLKIK-UHFFFAOYSA-N  InChI=1S/C16H15BrN2O3/c1-19-12-6-4-3-5-9(12)16...               COc1cc(Br)c(cc1O)C3N=C(O)c2ccccc2N3C  65.29  4.178839   
10002  JCP2022_010007  CBFRWEVECNRPOB-UHFFFAOYSA-N  InChI=1S/C22H23FN4O2/c23-17-7-11-19(12-8-17)29...     O=C(NCCC1=NN=C2CCCCCN12)c3ccc(cc3)Oc4ccc(F)cc4  69.04  4.234686   
10003  JCP2022_010008  CBFUDFNIKRRVDP-UHFFFAOYSA-N  InChI=1S/C16H20N2O2/c1-15(2)10(8-9-16(15,3)14(...               CC1(C)C(CCC1(C)C(O)=O)C3Nc2ccccc2N=3  65.98  4.189352   
10004  JCP2022_010009  CBFVKKYBQHBJCR-UHFFFAOYSA-N  InChI=1S/C19H19N3O2S/c1-12-15-18(21-14-10-6-3-...       CC2=C(SC3N=C1CCCCCN1C(=O)C2

In [27]:
print(results[1][2])

             JCP2022                                   pharmacophore_fp
0     JCP2022_010005  [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...
1     JCP2022_010006  [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
2     JCP2022_010007  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
3     JCP2022_010008  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, ...
4     JCP2022_010009  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
...              ...                                                ...
9995  JCP2022_020005  [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ...
9996  JCP2022_020006  [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
9997  JCP2022_020007  [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
9998  JCP2022_020008  [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...
9999  JCP2022_020009  [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...

[10000 rows x 2 columns]


## combine metadata files from all processes and write output 

- write file (updated with Pharmacophores) to CSV `./output/cp_compound_metadata_clean.csv`

In [84]:
print(f" compound Metadata + Base64 Pharmacophores File: {compoundMetadataPharmacophoreFile}")
print(f" compound Pharmacophore File                   : {compoundPharmacophoreFile}")


 compound Metadata + Base64 Pharmacophores File: ./output_09112023/cp_compound_metadata_pcfp.csv
 compound Pharmacophore File                   : ./output_09112023/cp_compound_pcfp_sparse.pkl


In [ ]:
# for i, res in enumerate(results):
#     chunk_id, df_chunk ,df_fingerprints = res    
#     mode = 'w' if chunk_id == 0 else 'a'
#     header = chunk_id == 0
#     print(f" {datetime.now().strftime('%X.%f')} | chunk_id: {chunk_id} | write to compoundDataOutputFile     | mode : {mode}  header: {header}")
#     df_chunk.to_csv( 
#                     index=False,  
#                     header=header, mode=mode) # Skip index column    

In [30]:
df_metadata_all = pd.concat([i[1] for i in results ], axis = 0, ignore_index=True )

In [32]:
# df_metadata_all.drop(columns='index', inplace = True)
df_metadata_all.info()
df_metadata_all.head()


df_temp = df_metadata_all[df_metadata_all['smiles'].isna()]
print(f" Smiles entries == NaN: {len(df_temp)}")
df_temp = df_metadata_all[df_metadata_all['InChI'].isna()]
print(f" InChI entries == NaN: {len(df_temp)}")
df_temp = df_metadata_all[df_metadata_all['pharmacophore_base64'].isna()]
print(f" Base64 Pharmcophore == NaN: {len(df_temp)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   JCP2022               60000 non-null  object 
 1   InChIKey              60000 non-null  object 
 2   InChI                 60000 non-null  object 
 3   smiles                60000 non-null  object 
 4   TPSA                  60000 non-null  float64
 5   lnTPSA                60000 non-null  float64
 6   log10TPSA             60000 non-null  float64
 7   pharmacophore_base64  60000 non-null  object 
dtypes: float64(3), object(5)
memory usage: 3.7+ MB


,JCP2022,InChIKey,InChI,smiles,TPSA,lnTPSA,log10TPSA,pharmacophore_base64
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...,63.49,4.150882,1.802705,4P///wgEAABaAAAAAgAAAAAAAAAABAAEAAAAAAACBAAIAA...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,OC1=NCCCN1Cc2ccc(Cl)cc2,35.83,3.578786,1.554247,4P///wgEAAAXAAAABgAEAhAAFAIAAAAO0GoADEixALUAAA...
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4,73.72,4.300274,1.867585,4P///wgEAABeAAAAAgAAAgAAAAAMAAAAAgAADgAAAAAMBA...
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,CCCN1N=CC=C1S(=O)(=O)N2CC3CNCC2CC3,67.23,4.208120,1.827563,4P///wgEAAAfAAAAAAAABAIKCgAACggUCDoCFAYGBiYGJg...
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC3N=C(c1ccc(Cl)cc1)c2cc(ccc2N4C(C)=NN...,84.89,4.441356,1.928857,4P///wgEAABhAAAAAAAAAAIAAAAADAACAAAAABAAAgwAAA...


 Smiles entries == NaN: 0
 InChI entries == NaN: 0
 Base64 Pharmcophore == NaN: 0


In [33]:
 
print(f" compound Metadata + Base64 Pharmacophores File: {compoundMetadataPharmacophoreFile}")

 compound Metadata Output File 3: ./output_09112023/cp_compound_metadata_pcfp.csv

 compound Metadata + Base64 Pharmacophores File: ./output_09112023/cp_compound_metadata_pcfp.csv


In [34]:
df_metadata_all.to_csv(compoundMetadataPharmacophoreFile, header = True, index = False)

## combine pharmacophore files from all processes and write output

- write to `./output/cp_compound_pharmacophores.pkl`

In [ ]:
#     print(f" {datetime.now().strftime('%X.%f')} | chunk_id: {chunk_id} | write to compoundPharmacophoreFile  | mode : {mode}  header: {header}")
#     df_fingerprints.to_csv(compoundPharmacophoreFile,
#                     index=False,  
#                     header=header, mode=mode)     # Skip index column

In [35]:
df_fingerprints_all = pd.concat([i[2] for i in results ], axis = 0, ignore_index=True )
df_fingerprints_all.info()
df_fingerprints_all.head()
df_fingerprints_all[df_fingerprints_all['pharmacophore_fp'].isna()]
len(df_fingerprints_all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JCP2022           60000 non-null  object
 1   pharmacophore_fp  60000 non-null  object
dtypes: object(2)
memory usage: 937.6+ KB


,JCP2022,pharmacophore_fp
0,JCP2022_000001,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ..."
1,JCP2022_000002,"[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
2,JCP2022_000003,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
3,JCP2022_000004,"[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,JCP2022_000005,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


,JCP2022,pharmacophore_fp


60000

In [41]:
print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")

 compound Pharmacophore File    : ./output_09112023/cp_compound_pcfp_sparse.pkl


In [42]:
df_fingerprints_all.to_pickle(compoundPharmacophoreFile)

#### Test for an all zero pharmacophore fp

In [48]:
df_fingerprints_all.iloc[0][0]
df_fingerprints_all.iloc[0][1]

'JCP2022_000001'

In [ ]:
# import rdkit

# i = 0
# zero_ctr = 0 
# all_zeros = [0]*1032
# test = [1]
# test.extend([0] *1031)

# print(all_zeros)
# print(test)
# print(all_zeros == test)

# for comp in df_fingerprints_all.itertuples():
#     list_fromBase64Sparse = rdkit.DataStructs.cDataStructs.SparseBitVect.ToList(df_fingerprints_all.iloc[0][1])
#     if list_fromBase64Sparse == all_zeros:
#         print(i, ' is all zeros ' , len(list_fromBase64Sparse))
#         zero_ctr += 1
#     i+=1
#     # if i == 20000:
#         # break
# print(f" number of records read {i}")
# print(f" number of zero fingerprints: {zero_ctr}")

In [43]:
# df_chunk.info()
# df_fingerprints.info()
del df_fingerprints_all,   df_metadata_all

## Reread and verify saved files

### Reread and check metadata (+ pharmacophore in base 64 format) CSV file 

- `./output/cp_compound_metadata_clean.csv`

In [63]:
print(f" compound Metadata Output File 3: {compoundMetadataOutputFile_3}\n")
# ./output/cp_compound_metadata_clean.csv

 compound Metadata Output File 2: ./output/cp_compound_metadata_clean.csv



In [64]:
df = pd.read_csv(compoundMetadataOutputFile_3)
df.info()
df.head(10)
# df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   JCP2022               116751 non-null  object
 1   InChIKey              116751 non-null  object
 2   InChI                 116751 non-null  object
 3   smiles                116751 non-null  object
 4   pharmacophore_base64  116751 non-null  object
dtypes: object(5)
memory usage: 4.5+ MB


,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...,4P///wgEAABaAAAAAgAAAAAAAAAABAAEAAAAAAACBAAIAA...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,OC1=NCCCN1Cc2ccc(Cl)cc2,4P///wgEAAAXAAAABgAEAhAAFAIAAAAO0GoADEixALUAAA...
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4,4P///wgEAABeAAAAAgAAAgAAAAAMAAAAAgAADgAAAAAMBA...
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,CCCN1N=CC=C1S(=O)(=O)N2CC3CNCC2CC3,4P///wgEAAAfAAAAAAAABAIKCgAACggUCDoCFAYGBiYGJg...
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC3N=C(c1ccc(Cl)cc1)c2cc(ccc2N4C(C)=NN...,4P///wgEAABhAAAAAAAAAAIAAAAADAACAAAAABAAAgwAAA...
5,JCP2022_000006,AAAROXVLYNJINN-UHFFFAOYSA-N,"InChI=1S/C16H20N6O/c1-16(2,3)22-13(10-5-6-10)7...",CN1C=C(C=N1)C2=NOC(=N2)C4C=C(C3CC3)N(N=4)C(C)(C)C,4P///wgEAABHAAAAAAAAAAQAAAAMAAQAAAAmAg4wAAACBg...
6,JCP2022_000007,AAATVJFGQAZXQO-UHFFFAOYSA-N,InChI=1S/C24H22N2O4S/c1-14-20(15(2)30-26-14)13...,CC(OC(=O)c1ccccc1SCC2C(C)=NOC=2C)C(=O)C3=CNc4c...,4P///wgEAABiAAAAAAAAAAIAAAAADgAAAAACEAAAAAwAAA...
7,JCP2022_000008,AAAUIKUIDRZZQT-UHFFFAOYSA-N,InChI=1S/C26H24ClN3O2/c1-3-22(25(31)29-20-13-9...,CCC(C(O)=Nc1cccc(Cl)c1)N4C(=O)C(C)N=C(c2ccccc2...,4P///wgEAAA9AAAAAgICAgIMAAIAAAAAEgIMAAAADABCHg...
8,JCP2022_000009,AAAZRMGPBSWFDK-UHFFFAOYSA-N,"InChI=1S/C18H19F3N4O4S2/c19-18(20,21)13-3-1-2-...",NC(=O)C1C=CSC=1N=C(O)CN2CCN(CC2)S(=O)(=O)c3ccc...,4P///wgEAABKAAAAAAAAAAAAAAAAAAYIAA4GAAAAAAYEDA...
9,JCP2022_000010,AABLKAPUNFMSIA-UHFFFAOYSA-N,InChI=1S/C15H24N4O4S/c1-3-18-14(4-7-16-18)15(2...,CCN1N=CC=C1C(=O)N2CC(C2)N(C3CCOCC3)S(C)(=O)=O,4P///wgEAAAPAAAAAAAAAAAAGHYIBAgEAGAKxQs=


In [65]:
df[df['InChI'].isna()]
len(df)
df[df['smiles'].isna()]
len(df)
df[df['pharmacophore_base64'].isna()]
len(df)


,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

In [47]:
del df 

### Reread and check pharmacophore fingerprint file 

- `./output/cp_compound_pharmacophores.pkl`

In [54]:
print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")

 compound Pharmacophore File    : ./output/cp_compound_pharmacophores.pkl


In [44]:
df = pd.read_pickle(compoundPharmacophoreFile)
df.info()
df.head(10)
df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   JCP2022           116751 non-null  object
 1   pharmacophore_fp  116751 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


,JCP2022,pharmacophore_fp
0,JCP2022_000001,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ..."
1,JCP2022_000002,"[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
2,JCP2022_000003,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
3,JCP2022_000004,"[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,JCP2022_000005,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
5,JCP2022_000006,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
6,JCP2022_000007,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
7,JCP2022_000008,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
8,JCP2022_000009,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ..."
9,JCP2022_000010,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


,JCP2022,pharmacophore_fp
116741,JCP2022_116744,"[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116742,JCP2022_116745,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116743,JCP2022_116746,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116744,JCP2022_116747,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
116745,JCP2022_116748,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116746,JCP2022_116749,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ..."
116747,JCP2022_116750,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ..."
116748,JCP2022_116751,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116749,JCP2022_116752,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116750,JCP2022_116753,"[0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, ..."


In [45]:

print(type(df.at[0, 'pharmacophore_fp']), len(df.at[0, 'pharmacophore_fp']),  df.at[0, 'pharmacophore_fp'])
print(type(df.loc[0].pharmacophore_fp)  , len(df.loc[0].pharmacophore_fp)  ,  df.loc[0].pharmacophore_fp)
print(type(df.iloc[0].pharmacophore_fp) , len(df.iloc[0].pharmacophore_fp) ,  df.iloc[0].pharmacophore_fp)
# print(type(df[0].pharmacophore_fp) , len(df[0].pharmacophore_fp) ,  df[0].pharmacophore_fp)


<class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 1032 <rdkit.DataStructs.cDataStructs.SparseBitVect object at 0x7fc484318f90>
<class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 1032 <rdkit.DataStructs.cDataStructs.SparseBitVect object at 0x7fc484318f90>
<class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 1032 <rdkit.DataStructs.cDataStructs.SparseBitVect object at 0x7fc484318f90>


# Convert RDKit Sparse Bit Vectors to dense matrix

In [126]:
from utils import fingerprint_to_sparse_matrix

In [67]:
print(f" compound Pharmacophore Sparse File       : {compoundPharmacophoreFile}")
print(f" compound Pharmacophore Sparse Clean File : {compoundPharmacophoreCleanFile}")
print(f" compound Pharmacophore Dense File        : {compoundPharmacophoreDenseFile}")
print(f" compound Pharmacophore Dense Zipped File : {compoundPharmacophoreDenseZipFile}")

 compound Pharmacophore Sparse File       : ./output_09112023/cp_compound_pcfp_sparse.pkl
 compound Pharmacophore Sparse Clean File : ./output_09112023/cp_compound_pcfp_sparse_clean.pkl
 compound Pharmacophore Dense File        : ./output_09112023/cp_compound_pcfp_dense.npy
 compound Pharmacophore Dense Zipped File : ./output_09112023/cp_compound_pcfp_dense.npz


In [70]:
# del df
df = pd.read_pickle(compoundPharmacophoreFile)


In [83]:
df.info()
df.head(3)
df.tail(3)
type(df.loc[0][1])

df_nan = df[df['pharmacophore_fp'].isna()]
len(df_nan)

df.loc[20:26]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   JCP2022           60000 non-null  object
 1   pharmacophore_fp  60000 non-null  object
dtypes: object(2)
memory usage: 937.6+ KB


,JCP2022,pharmacophore_fp
0,JCP2022_000001,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ..."
1,JCP2022_000002,"[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
2,JCP2022_000003,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


,JCP2022,pharmacophore_fp
59997,JCP2022_060029,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ..."
59998,JCP2022_060030,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
59999,JCP2022_060031,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ..."


rdkit.DataStructs.cDataStructs.SparseBitVect

0

,JCP2022,pharmacophore_fp
20,JCP2022_000021,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
21,JCP2022_000022,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, ..."
22,JCP2022_000023,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
23,JCP2022_000024,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
24,JCP2022_000025,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
25,JCP2022_000026,"[0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
26,JCP2022_000027,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ..."


In [74]:
start, end  = 0, 0 

for i in range(0, 6, 1):
    start = 10000 * i 
    end   +=10000
    print(f" {datetime.now().strftime('%X.%f')} | from {start} to {end}")
    csr_out, output_out = fingerprint_to_sparse_matrix(df, start, end, ncols=1032)
    if i == 0:
        dense_mat = csr_out.todense();
    else :
        dense_mat = np.concatenate([dense_mat, csr_out.todense()]);
    print(f" {datetime.now().strftime('%X.%f')} | shape of dense matrix is : {dense_mat.shape}" )

 15:06:33.960633 | from 0 to 10000
 15:06:38.775651 | shape of dense matrix is : (10000, 1032)
 15:06:38.775775 | from 10000 to 20000
 15:06:43.416728 | shape of dense matrix is : (20000, 1032)
 15:06:43.416872 | from 20000 to 30000
 15:06:48.142801 | shape of dense matrix is : (30000, 1032)
 15:06:48.142959 | from 30000 to 40000
 15:06:52.836535 | shape of dense matrix is : (40000, 1032)
 15:06:52.836687 | from 40000 to 50000
 15:06:57.543181 | shape of dense matrix is : (50000, 1032)
 15:06:57.543303 | from 50000 to 60000
 15:07:02.234859 | shape of dense matrix is : (60000, 1032)


In [75]:
print(f" Dense matrix - shape: {dense_mat.shape}  type:  {type(dense_mat)}")
# del dense_mat,csr_out,csr_matrix

 Dense matrix - shape: (60000, 1032)  type:  <class 'numpy.matrix'>


In [76]:
print(f" compound Pharmacophore Dense File : {compoundPharmacophoreDenseFile}")
print(f" compound Pharmacophore Dense Zipped File : {compoundPharmacophoreDenseZipFile}")

 compound Pharmacophore Dense File : ./output_09112023/cp_compound_pcfp_dense.npy
 compound Pharmacophore Dense Zipped File : ./output_09112023/cp_compound_pcfp_dense.npz


In [77]:
np.save(compoundPharmacophoreDenseFile, dense_mat)

In [78]:
np.savez_compressed(compoundPharmacophoreDenseZipFile, dense_mat = dense_mat)

# Reread and verify zipped dense matrix

In [79]:
loaded = np.load(compoundPharmacophoreDenseZipFile)
print(np.array_equal(dense_mat, loaded['dense_mat']))

True


In [80]:
# print(output_out)
type(loaded['dense_mat']), loaded['dense_mat'].shape
# loaded['dense_mat']

(numpy.ndarray, (60000, 1032))

In [81]:
del loaded